# Chicago Public Official Overtime Analysis

This notebook pulls data from the Chicago Data Portal and persists it in a SQLite Database

This work is split into an additional notebook.  This data was also analyzed with Airbnb's Superset

### Load dependencies

In [1]:
import pandas as pd
import sqlite3


---

## Data ingestion
---

### Download Links:

In [64]:
ot_download_links = {
    2016:"https://data.cityofchicago.org/api/views/tsqz-67gi/rows.csv?accessType=DOWNLOAD",
    2015:"https://data.cityofchicago.org/api/views/efd5-hyfi/rows.csv?accessType=DOWNLOAD",
    2014:"https://data.cityofchicago.org/api/views/9xua-tabs/rows.csv?accessType=DOWNLOAD",
    2013:"https://data.cityofchicago.org/api/views/s9zb-nuc6/rows.csv?accessType=DOWNLOAD",
    2012:"https://data.cityofchicago.org/api/views/92xk-4rg9/rows.csv?accessType=DOWNLOAD"
}

### Download data from each url - store as Pandas DataFrames

Stored as dictionary:

`{df_name: DataFrame}`

This is one way to work with multiple dataframes

In [65]:
ot_data = {} #overtime data frames
for year in ot_download_links.keys():
    ot_data['ot_'+ str(year)] = pd.read_csv(ot_download_links[year])
    print("Shape of OT data for "+ str(year)+": " + str(ot_data['ot_'+ str(year)].shape))

Shape of OT data for 2016: (25764, 16)
Shape of OT data for 2012: (24270, 16)
Shape of OT data for 2013: (22708, 16)
Shape of OT data for 2014: (24273, 16)
Shape of OT data for 2015: (24082, 16)


# Overtime Data 
---


In [66]:
ot_data['ot_2012'].head(1)

,Department Name,Employee Name,Title,Jan 2012,Feb 2012,Mar 2012,Apr 2012,May 2012,Jun 2012,Jul 2012,Aug 2012,Sept 2012,Oct 2012,Nov 2012,Dec 2012,2012 Total
0,Administrative Hearings,"WOODRIDGE, RACHENETTE",ADMINISTRATIVE ASST II,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$445.00,$0.00,$445.00


#### Check out headers

This is the same data set from 5 years, I wanted to see if the schema was consistent.  The naming conventions changed, but the order of column values was consistent over the overs.  

- If there are more years to analyze, write a function to compare column names and/or dtypes to see if schema changes over the years

In [67]:
for file_name in ot_data.keys():
    print( ot_data[file_name].columns)

Index([u'Department Name', u'Employee Name', u'Title', u'Jan 2012',
       u'Feb 2012', u'Mar 2012', u'Apr 2012', u'May 2012', u'Jun 2012',
       u'Jul 2012', u'Aug 2012', u'Sept 2012', u'Oct 2012', u'Nov 2012',
       u'Dec 2012', u'2012 Total'],
      dtype='object')
Index([u'DEPARTMENT NAME', u'EMPLOYEE NAME', u'TITLE', u'JAN 2013',
       u'FEB 2013', u'MAR 2013', u'APR 2013', u'MAY 2013', u'JUN 2013',
       u'JUL 2013', u'AUG 2013', u'SEP 2013', u'OCT 2013', u'NOV 2013',
       u'DEC 2013', u'2013 TOTAL'],
      dtype='object')
Index([u'DEPARTMENT NAME', u'NAME', u'TITLE', u'JANUARY', u'FEBRUARY',
       u'MARCH', u'APRIL', u'MAY', u'JUNE', u'JULY', u'AUGUST', u'SEPTEMBER',
       u'OCTOBER', u'NOVEMBER', u'DECEMBER', u'TOTAL'],
      dtype='object')
Index([u'DEPARTMENT NAME', u'EMPLOYEE NAME', u'TITLE', u'JANUARY', u'FEBRARY',
       u'MARCH', u'APRIL', u'MAY', u'JUNE', u'JULY', u'AUGUST', u'SEPTEMBER',
       u'OCTOBER', u'NOVEMBER', u'DECEMBER', u'TOTAL'],
      dtype='object

### Clean data

- Found the numbers were stored as strings.  
- Convert the strings to floats, don't convert the actual string columns, print the name of the column if can't convert for some reason.

In [68]:
def ot_clean(month_df):
    for col in month_df.columns.tolist():
        if col.upper() not in ['DEPARTMENT NAME', 'EMPLOYEE NAME', 'TITLE', 'NAME']:
            try:
                month_df[col] = month_df[col].str.replace(',', '')
                month_df[col] = month_df[col].str.replace('$', '')
                month_df[col] = month_df[col].astype(float)
            except: 
                print(col)
    return(month_df)

In [69]:
for file_name in ot_data.keys():
    try:
        ot_data[file_name] = ot_clean(ot_data[file_name])
    except:
        print (file_name)

### Add a column for the year for storage

In [70]:
from datetime import datetime

#### Fixing my mistake - found when investigating DB data.

In [71]:
# for file_name in ot_data.keys():
#     ot_data[file_name].drop(['Year','year'], axis=1, inplace=True)

In [72]:
for file_name in ot_data.keys():
    ot_data[file_name]['year'] = datetime.strptime('12/31/'+ str(file_name[-4:]), '%m/%d/%Y')

### Create DB + Add our data

In [90]:
def create_db(db_path):
    '''Bad function design'''
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    try:    
        cur.execute("DROP TABLE IF EXISTS ot")#drop if already exists tables in DB
        cur.execute("""CREATE TABLE ot
           (dept_name text, emp_name text, title text, jan numeric, feb numeric,
       mar numeric, apr numeric, may numeric, jun numeric,jul numeric, aug numeric, sep numeric,
       oct numeric, nov numeric, dec numeric, total numeric, year text)
        """)
    except Exception as e:
        print(e)
    con.commit()
    con.close()

Need consistent column names for all the pandas dataframes.  We'll be pushing dataframes directly to sqlite.


In [91]:
table_cols = ['dept_name' , 'emp_name', 'title' , 'jan' , 'feb' ,'mar' , 'apr' , 'may' , 'jun' ,'jul' , 'aug' , 'sep' , 'oct' , 'nov' , 'dec' , 'total' , 'year']

In [92]:
db_path = 'chicago_data.db'
create_db(db_path)

In [93]:
con = sqlite3.connect(db_path)#Create database to store movie and rating info for easy storage and querying
cur = con.cursor()

#### One last check on the data before adding to DB

In [94]:
ot_data['ot_2012'].head()

,dept_name,emp_name,title,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,total,year
0,Administrative Hearings,WOODRIDGE RACHENETTE,ADMINISTRATIVE ASST II,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,445.0,0.0,445.0,2012-12-31
1,Animal Care and Control,HOLLOWAY EFREM J,ANIMAL CARE AIDE II,278.0,670.0,1040.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,287.0,2275.0,2012-12-31
2,Animal Care and Control,EDGECOMBE CHERYL K,ANIMAL CONTROL OFFICER,404.0,0.0,771.0,0.0,147.0,637.0,1114.0,478.0,257.0,539.0,257.0,514.0,5118.0,2012-12-31
3,Animal Care and Control,RUGLIC MARY S,ANIMAL CONTROL OFFICER,0.0,147.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,147.0,2012-12-31
4,Animal Care and Control,WASHINGTON MONTESE DAJUAN,VETERINARY ASST,575.0,780.0,1135.0,492.0,657.0,913.0,1552.0,821.0,780.0,954.0,855.0,472.0,9985.0,2012-12-31


#### Loop through my dictionary keys, using:    `Pandas.DataFrame.to_sql(table_name, connection, ...)`

In [95]:
for file_name in ot_data.keys():
    ot_data[file_name].columns = table_cols
    ot_data[file_name].to_sql('ot', con, if_exists='append', index=False)

In [96]:
cur.execute("""SELECT COUNT(*) FROM ot""")

In [97]:
cur.fetchall()

[(121097,)]

### 'ot' Table Column Names:

- 'dept_name'
- 'emp_name'
- 'title'
- 'jan'
- 'feb'
- 'mar'
- 'apr'
- 'may'
- 'jun'
- 'jul'
- 'aug' 
- 'sep'
- 'oct'
- 'nov'
- 'dec'
- 'total' 
- 'year'

In [28]:
cur.execute("""SELECT total, year 
                FROM ot
                GROUP BY year""")

In [29]:
q = """SELECT total, year 
                FROM ot
                GROUP BY year"""

In [30]:
df = pd.read_sql(q,con)

In [31]:
df.head()

,total,year
0,15554.00,2012
1,609.30,2013
2,50211.49,2014
3,22773.31,2015
4,25249.60,2016


In [32]:
cur.close()

Dollar values are looking like strings, we'll need to fix that.  I also don't care about about names for now, more concerned with Department Name.  Title could be interesting for other analysis but lets start with analysis grouped by Department.

---

# OT Data - Normalize
---

Restructure data

In [73]:
ot_data_original = ot_data

In [74]:
keep_list = ot_data['ot_2012'].columns.tolist()[3:-2]

In [75]:
[x.upper() for x in ot_data['ot_2012'].columns.tolist()]

['DEPARTMENT NAME',
 'EMPLOYEE NAME',
 'TITLE',
 'JAN 2012',
 'FEB 2012',
 'MAR 2012',
 'APR 2012',
 'MAY 2012',
 'JUN 2012',
 'JUL 2012',
 'AUG 2012',
 'SEPT 2012',
 'OCT 2012',
 'NOV 2012',
 'DEC 2012',
 '2012 TOTAL',
 'YEAR']

In [77]:
work_df = pd.melt(ot_data['ot_2012'], id_vars=['Department Name','Title'], value_vars=keep_list)

In [78]:
work_df.columns

Index([u'Department Name', u'Title', u'variable', u'value'], dtype='object')

In [79]:
work_df.columns = ['Department Name', 'Title', 'Date', 'OT Pay']

In [80]:
work_df.head()

,Department Name,Title,Date,OT Pay
0,Administrative Hearings,ADMINISTRATIVE ASST II,Jan 2012,0.0
1,Animal Care and Control,ANIMAL CARE AIDE II,Jan 2012,278.0
2,Animal Care and Control,ANIMAL CONTROL OFFICER,Jan 2012,404.0
3,Animal Care and Control,ANIMAL CONTROL OFFICER,Jan 2012,0.0
4,Animal Care and Control,VETERINARY ASST,Jan 2012,575.0


In [81]:
def clean_date(date):
    date = date.split(' ')[0][:3] + " " + date.split(' ')[1]
    return(datetime.strptime(date , '%b %Y'))

In [82]:
work_df['Date'] = work_df['Date'].map(lambda x: clean_date(x))

In [83]:
work_df.head()

,Department Name,Title,Date,OT Pay
0,Administrative Hearings,ADMINISTRATIVE ASST II,2012-01-01,0.0
1,Animal Care and Control,ANIMAL CARE AIDE II,2012-01-01,278.0
2,Animal Care and Control,ANIMAL CONTROL OFFICER,2012-01-01,404.0
3,Animal Care and Control,ANIMAL CONTROL OFFICER,2012-01-01,0.0
4,Animal Care and Control,VETERINARY ASST,2012-01-01,575.0


In [84]:
def clean_date(date, name = None):
    if ' ' in date:
        date = date.split(' ')[0][:3] + " " + date.split(' ')[1]
    else:
        date = date[:3] + " " + name[-4:]
    return(datetime.strptime(date , '%b %Y'))

def prep_long_data(df, name = None):
    print('Prepping colulmns...')
    df.columns = [x.upper() for x in df.columns.tolist()]
    
    keep_list = df.columns.tolist()[3:-2]
    print('Reshaping data...')
    temp_df = pd.melt(df, id_vars=['DEPARTMENT NAME','TITLE'], value_vars=keep_list)
    
    temp_df.columns = ['Department Name', 'Title', 'Date', 'OT Pay']
    print('Converting date...')
    temp_df['Date'] = temp_df['Date'].map(lambda x: clean_date(x, name))
    print(name+ ' Complete! \n')
    return(temp_df)

In [85]:
test = prep_long_data(ot_data['ot_2015'], 'ot_2015')

Prepping colulmns...
Reshaping data...
Converting date...
ot_2015 Complete! 



In [86]:
for df_name in ot_data.keys():
    ot_data[df_name]= prep_long_data(ot_data[df_name], df_name)

Prepping colulmns...
Reshaping data...
Converting date...
ot_2012 Complete! 

Prepping colulmns...
Reshaping data...
Converting date...
ot_2013 Complete! 

Prepping colulmns...
Reshaping data...
Converting date...
ot_2016 Complete! 

Prepping colulmns...
Reshaping data...
Converting date...
ot_2014 Complete! 

Prepping colulmns...
Reshaping data...
Converting date...
ot_2015 Complete! 



In [95]:
table_cols = ['dept_name', 'title', 'date', 'ot_pay']

In [98]:
def create_db(db_path):
    '''Bad function design'''
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    try:    
        cur.execute("DROP TABLE IF EXISTS ot_long")#drop if already exists tables in DB
        cur.execute("""CREATE TABLE ot_long
           (dept_name text, title text, date text, ot_pay numeric)
        """)
    except Exception as e:
        print(e)
    con.commit()
    con.close()

In [99]:
db_path = 'chicago_data.db'
create_db(db_path)

In [100]:
con = sqlite3.connect(db_path)#Create database to store movie and rating info for easy storage and querying
cur = con.cursor()

#### One last check on the data before adding to DB

In [101]:
ot_data['ot_2012'].head()

,Department Name,Title,Date,OT Pay
0,Administrative Hearings,ADMINISTRATIVE ASST II,2012-01-01,0.0
1,Animal Care and Control,ANIMAL CARE AIDE II,2012-01-01,278.0
2,Animal Care and Control,ANIMAL CONTROL OFFICER,2012-01-01,404.0
3,Animal Care and Control,ANIMAL CONTROL OFFICER,2012-01-01,0.0
4,Animal Care and Control,VETERINARY ASST,2012-01-01,575.0


#### Loop through my dictionary keys, using:    `Pandas.DataFrame.to_sql(table_name, connection, ...)`

In [102]:
for file_name in ot_data.keys():
    ot_data[file_name].columns = table_cols
    ot_data[file_name].to_sql('ot_long', con, if_exists='append', index=False)

In [105]:
cur.execute("""SELECT COUNT(*) FROM ot_long""")

In [106]:
cur.fetchall()

[(1453164,)]

In [107]:
cur.close()
con.close()